In [ ]:
!pip install --upgrade pip
!pip install transformers
!pip install simpletransformers
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize

In [2]:
import numpy as np 
import pandas as pd 
import os 
from tqdm import tqdm
import warnings
warnings.simplefilter('ignore')
import gc
from scipy.special import softmax
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from keras.utils import to_categorical
import random
import torch
import logging
from google.colab import drive

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [8]:
# drive.mount('/content/gdrive')
INPUT_DIR = '/content/drive/My Drive/'

# Suppress warnings in cells to improve readability
import warnings  
warnings.filterwarnings('ignore')

In [9]:
#choose the same seed to assure that our model will be roproducible
seed_value = 666
CUDA = False

def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        CUDA = True
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False
    return CUDA

CUDA = seed_all(seed_value)

In [17]:
CUDA

True

In [10]:
train = pd.read_csv(INPUT_DIR + "Quotes.csv", delimiter=';', header=None, names=['quote', 'author', 'label'])
feat_cols = "quote"
train = train.dropna()

In [11]:
all_labels = train.label.unique()
keep = ['amazing','art', 'attitude', 'beauty', 'best','business','change','communication','cool', 'courage','design','dreams', 'education','equality', 'experience', 'failure', 'family', 'famous', 'fear', 'finance', 'fitness', 'forgiveness', 'freedom', 'friendship', 'future',
       'good','great','happiness', 'health','hope','imagination', 'inspirational', 'intelligence','knowledge', 'leadership', 'learning','life', 'love',
       'money','motivational','movingon','patience','positive', 'power', 'relationship','respect', 'smile', 'society','strength', 'success', 'sympathy',
       'thankful', 'time', 'trust','truth','wisdom','work']
drop = [i for i in all_labels if i not in keep]

for item in drop:
    train = train.drop(train.index[train.label == item])

group1 = ['art', 'design', 'education', 'learning', 'knowledge', 'intelligence']
group2 = ['attitude', 'best', 'amazing', 'cool', 'courage',   'good', 'great', 'happiness','hope',  'inspirational','motivational',  'patience', 'positive', 'power','respect', 'smile']
group3 = ['beauty', 'family', 'fitness', 'friendship', 'health', 'love','movingon', 'life','relationship', 'society', 'strength']
group4 = ['work', 'business', 'communication', 'finance', 'leadership', 'money', 'famous', 'success']
group5 = ['change', 'equality', 'fear', 'dreams', 'experience', 'failure', 'forgiveness', 'freedom','future', 'imagination',  'sympathy', 'thankful', 'time', 'trust', 'truth','wisdom']

# create a list of our conditions
conditions = [
    (train['label'].isin(group1)),
    (train['label'].isin(group2)),
    (train['label'].isin(group3)),
    (train['label'].isin(group4)),
    (train['label'].isin(group5))
    ]

# create a list of the values we want to assign for each condition
label_cols = ['thoughtful', 'emotional', 'personal', 'work', 'aspirations']

# create a new column and use np.select to assign values to it using our lists as arguments
train['label2'] = np.select(conditions, label_cols)

# display updated DataFrame
train.head()

,quote,author,label,label2
1997,Sometimes I think my husband is so amazing tha...,Angelina Jolie,amazing,emotional
1998,If you tell people where to go but not how to ...,George S. Patton,amazing,emotional
1999,It is amazing what you can accomplish if you d...,Harry S. Truman,amazing,emotional
2000,If you set goals and go after them with all th...,Les Brown,amazing,emotional
2001,Nobody in life gets exactly what they thought ...,Conan O'Brien,amazing,emotional


In [12]:
train.drop(columns="label", inplace=True)
train.drop(columns="author", inplace=True)
train = train.rename(columns={"label2": "label"})

In [13]:
# l=['thoughtful', 'emotional', 'personal', 'work', 'aspirations']

# Get the numerical ids of column label
train['label']=train.label.astype('category')

Y = train.label.cat.codes
train['label']=Y

# Print initial shape
print(Y.shape)

# One-hot encode the indexes
Y = to_categorical(Y)

# Check the new shape of the variable
print(Y.shape)

(42030,)
(42030, 5)


In [14]:
train.head()

,quote,label
1997,Sometimes I think my husband is so amazing tha...,1
1998,If you tell people where to go but not how to ...,1
1999,It is amazing what you can accomplish if you d...,1
2000,If you set goals and go after them with all th...,1
2001,Nobody in life gets exactly what they thought ...,1


In [15]:
X_train, X_test, y_train, y_test = train_test_split(train, train['label'], test_size=0.33, random_state=seed_value)
gc.collect()

11

### Default ###
args = {

    "output_dir": "outputs/",
    "cache_dir": "cache/",
    "best_model_dir": "outputs/best_model/",

    "fp16": True,
    "max_seq_length": 128,
    "train_batch_size": 8,
    "eval_batch_size": 8,
    "gradient_accumulation_steps": 1,
    "num_train_epochs": 1,
    "weight_decay": 0,
    "learning_rate": 4e-5,
    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,
    "do_lower_case": False,

    "logging_steps": 50,
    "evaluate_during_training": False,
    "evaluate_during_training_steps": 2000,
    "evaluate_during_training_verbose": False,
    "use_cached_eval_features": False,
    "save_eval_checkpoints": True
    "save_steps": 2000,
    "no_cache": False,
    "save_model_every_epoch": True,
    "tensorboard_dir": None,

    "overwrite_output_dir": False,
    "reprocess_input_data": True,

    "process_count": cpu_count() - 2 if cpu_count() > 2 else 1
    "n_gpu": 1,
    "silent": False,
    "use_multiprocessing": True,

    "wandb_project": None,
    "wandb_kwargs": {},

    "use_early_stopping": True,
    "early_stopping_patience": 3,
    "early_stopping_delta": 0,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize": True,

    "manual_seed": None,
    "encoding": None,
    "config": {},
}

In [16]:
model_args = {'reprocess_input_data': True,
             'overwrite_output_dir': True,
             'fp16': False,
             'manual_seed': seed_value}

In [18]:
%%time
model = ClassificationModel('roberta', 'roberta-base', use_cuda=CUDA, num_labels=len(label_cols), args=model_args)
model.train_model(X_train)
raw_outputs_val = model.eval_model(X_test)[1]  # result, model_outputs, wrong_predictions = model.eval_model(train1_val)
raw_outputs_vals = softmax(raw_outputs_val, axis=1)
print(f"Log_Loss: {log_loss(y_test, raw_outputs_vals)}")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out


Log_Loss: 0.34300370406837516
CPU times: user 10min 1s, sys: 4min 45s, total: 14min 47s
Wall time: 16min 16s


In [19]:
def estm(raw_outputs_vals):
    for i in range(len(raw_outputs_vals)):
        for j in range(len(label_cols)):
            if(max(raw_outputs_vals[i])==raw_outputs_vals[i][j]):
                raw_outputs_vals[i][j]=1
            else :
                raw_outputs_vals[i][j]=0
    return(raw_outputs_vals)

def get_ans(choice):
    labels = ['thoughtful', 'emotional', 'personal', 'work', 'aspirations']
    it = np.nditer(choice, flags=['f_index'])
    for x in it:
        if x == 1:
            ans = labels[it.index]
    return ans

In [20]:
gc.collect()

420

In [21]:
outputs_vals = estm(raw_outputs_vals)
outputs_vals

array([[0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [22]:
pred = model.predict(['It is amazing what you can accomplish if you do not care who gets the credit.'])[1] # predictions, raw_outputs = model.predict(["Some arbitary sentence"])
preds = softmax(pred, axis=1)
choice = estm(preds)
ans = get_ans(choice)
ans

'emotional'

In [27]:
# Saving the quantized model
model.save_model("/content/drive/My Drive/outputs/quantized", model=model.model)

In [28]:
# Load the saved (quantized) model
model2 = ClassificationModel('roberta', "/content/drive/My Drive/outputs/quantized", use_cuda=CUDA)

In [29]:
# The outputs for the loaded model should be identical to the original (after quantization) model
pred2 = model2.predict(["Nobody in life gets exactly what they thought they were going to get. But if you work really hard and you're kind amazing things will happen."])[1]
preds2 = softmax(pred2, axis=1)
choice2 = estm(preds2)
ans2 = get_ans(choice2)
ans2

'emotional'